# EV Range Prediction

In [3]:
import pandas as pd
import numpy as np
import requests
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib


df = pd.read_csv('ev_range_data_updated.csv')
df.head() 

,trip_distance,elevation_change,traffic_delay,battery_consumption,estimated_range
0,190.397359,-21.272896,1120.922455,42.619203,446.740774
1,475.603582,-5.312868,998.736289,96.848053,491.082233
2,367.337001,70.909479,528.461738,76.433793,480.595016
3,301.335950,-31.999123,1821.800010,62.556871,481.699203
4,82.229227,73.929937,1429.872482,21.901019,375.458448


In [4]:
df.columns

Index(['trip_distance', 'elevation_change', 'traffic_delay',
       'battery_consumption', 'estimated_range'],
      dtype='object')

## Data Preprocessing

In [5]:
df = df.dropna()
df = pd.get_dummies(df, drop_first=True)


X = df.drop(columns=['estimated_range'])  
y = df['estimated_range']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
print(X_test.shape)

(1000, 4)


# Fetch Real-time Terrain & Traffic Data


In [7]:

def get_elevation(lat, lon):
    url = f'https://api.opentopodata.org/v1/test-dataset?locations={lat},{lon}'
    response = requests.get(url).json()
    return response['results'][0]['elevation'] if 'results' in response else None



In [8]:
import requests

def get_route_info(start_lon, start_lat, end_lon, end_lat,api_key):
    url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={api_key}&start={start_lon},{start_lat}&end={end_lon},{end_lat}"
    
    response = requests.get(url).json()

    if 'features' in response and len(response['features']) > 0:
        route = response['features'][0]['properties']
        distance = (route['segments'][0]['distance'] )/1000# In meters
        duration = ((route['segments'][0]['duration'])/3600) # In seconds
        return distance, duration
    
    return None, None

# Train XGBoost Model with Hyperparameter Tuning

In [ ]:
params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror')
grid_search = GridSearchCV(xgb_model, params, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_

print('Best Model:', best_model)

Best Model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)


In [10]:
print("Input data shape:", X_train.shape)


Input data shape: (4000, 4)


Model Evaluation

In [33]:
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(f'MAE: {mae}, MSE: {mse}')

MAE: 24.27174825180004, MSE: 16906.277687811376


 Save the Model for Deployment

In [27]:
joblib.dump(best_model, 'ev_range_model.pkl')
print('Model saved as ev_range_model.pkl')

Model saved as ev_range_model.pkl
